# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [175]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [176]:
# Output File (CSV)
output_data_file = "./output_data/weather_df.csv"
weather_df = pd.read_csv(output_data_file, index_col=0)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [177]:
weather_df.columns

Index(['Unnamed: 0.1', 'City', 'Cloudiness', 'Country', 'Date', 'Humidity',
       'Lat', 'Lng', 'Max Temp', 'Wind Speed'],
      dtype='object')

In [178]:
# Store latitude and longitude in locations
locations = weather_df[['Lat','Lng']]


In [179]:
fig = gmaps.figure()

# Create heat layer, with humidity as weight 
heat_layer = gmaps.heatmap_layer(locations, weights=weather_df['Humidity'], dissipating=False, max_intensity=10,point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [180]:
criteria_df = weather_df[(weather_df['Max Temp']>=70) & (weather_df['Max Temp']<=80) & (weather_df['Wind Speed'] <=10) & (weather_df['Cloudiness']== 0) ]
# criteria_df.drop('Unnamed: 0', axis=1)
criteria_df


,Unnamed: 0.1,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
56,56,port alfred,0,ZA,1584193149,85,-33.59,26.89,72.00,4.00
79,79,molteno,0,ZA,1584193150,24,-31.40,26.36,76.55,7.25
208,208,langsa,0,ID,1584193158,84,4.47,97.97,78.17,3.65
222,222,awjilah,0,LY,1584193159,21,29.11,21.29,76.73,8.95
308,308,emerald,0,AU,1584193043,56,-23.53,148.17,71.60,8.05
338,338,montemor-o-novo,0,PT,1584193165,64,38.65,-8.21,79.00,4.70
409,409,rajpur,0,IN,1584193169,15,22.30,74.35,71.87,9.55
416,416,lata,0,PT,1584193170,63,40.16,-8.33,75.00,8.05
485,485,balaipungut,0,ID,1584193173,88,1.05,101.28,74.75,2.80
490,490,scicli,0,IT,1584193174,33,36.79,14.70,73.40,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [181]:

hotel_df = criteria_df.reset_index(drop=True)

hotel_df["Hotel Name"] = ""

params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_name = requests.get(base_url, params=params)
    
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

PqBjkNEhClGmT5-qHcg7MU_xNYYdc6GhSi_DNw_jP9Gz1CvxiYhEBfV9-c4w",
                    "width": 2048
                }
            ],
            "place_id": "ChIJHxLY_E-8-zoR-tPXKwfQj1Q",
            "plus_code": {
                "compound_code": "J689+25 Trincomalee, Sri Lanka",
                "global_code": "6MW3J689+25"
            },
            "rating": 4.2,
            "reference": "ChIJHxLY_E-8-zoR-tPXKwfQj1Q",
            "scope": "GOOGLE",
            "types": [
                "lodging",
                "point_of_interest",
                "establishment"
            ],
            "user_ratings_total": 284,
            "vicinity": "07(42) Alles Garden, Trincomalee"
        },
        {
            "geometry": {
                "location": {
                    "lat": 8.610962599999999,
                    "lng": 81.2151023
                },
                "viewport": {
                    "northeast": {
                        "lat": 8.61225637989272,
                     

In [182]:
hotel_

,Unnamed: 0.1,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,56,port alfred,0,ZA,1584193149,85,-33.59,26.89,72.00,4.00,The Beach House
1,79,molteno,0,ZA,1584193150,24,-31.40,26.36,76.55,7.25,Molteno Guesthouse
2,208,langsa,0,ID,1584193158,84,4.47,97.97,78.17,3.65,RedDoorz Syariah near Universitas Samudera Langsa
3,222,awjilah,0,LY,1584193159,21,29.11,21.29,76.73,8.95,
4,308,emerald,0,AU,1584193043,56,-23.53,148.17,71.60,8.05,Village National Pritchard Road Country Resort
5,338,montemor-o-novo,0,PT,1584193165,64,38.65,-8.21,79.00,4.70,Palacete da Real Companhia do Cacau - Royal Co...
6,409,rajpur,0,IN,1584193169,15,22.30,74.35,71.87,9.55,Hotel sai kripa
7,416,lata,0,PT,1584193170,63,40.16,-8.33,75.00,8.05,
8,485,balaipungut,0,ID,1584193173,88,1.05,101.28,74.75,2.80,Swee Ann Homestay
9,490,scicli,0,IT,1584193174,33,36.79,14.70,73.40,9.17,PALAZZO MONTALBANO


In [183]:
# NOTE: Do not change any of the code in this cell
# NOTE: At the bottom of this cell you ask us to change the code in this cell 
 
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [184]:

# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)

# Display Map
fig


Figure(layout=FigureLayout(height='420px'))